In [99]:
#pip install selenium

In [100]:
from selenium import webdriver #The Selenium package we'll need
import time #This package lets us pause execution for a bit
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [101]:
url = 'https://www.tripadvisor.com/Restaurants-g189473-Thessaloniki_Thessaloniki_Region_Central_Macedonia.html'

In [102]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)
driver.find_element(By.ID, "onetrust-accept-btn-handler").click()


In [103]:
def selectCheckbox1(showMore, driver):
    showMore.click()

    time.sleep(1)
    label_text = "Coffee & Tea"
    label_element = driver.find_element(By.XPATH, "//label[contains(., '{}')]".format(label_text))
    checkbox = label_element.find_element(By.XPATH, "./preceding-sibling::input")
    driver.execute_script("arguments[0].click();", checkbox)

    time.sleep(1)
    label_text = "Bars & Pubs"
    label_element = driver.find_element(By.XPATH, "//label[contains(., '{}')]".format(label_text))
    checkbox = label_element.find_element(By.XPATH, "./preceding-sibling::input")
    driver.execute_script("arguments[0].click();", checkbox)

    time.sleep(3)
    label_text = "Restaurants"
    label_element = driver.find_elements(By.XPATH, "//label[contains(., '{}')]".format(label_text))
    checkbox = label_element[2].find_element(By.XPATH, "./preceding-sibling::input")
    driver.execute_script("arguments[0].removeAttribute('checked'); arguments[0].click();", checkbox)
        

In [104]:
def selectCheckbox2(showMore, driver):
    showMore.click()

    label_text = "Coffee & Tea"
    label_element = driver.find_element(By.XPATH, "//label[contains(., '{}')]".format(label_text))
    checkbox = label_element.find_element(By.XPATH, "preceding::input[1]")
    driver.execute_script("arguments[0].click();", checkbox)


    time.sleep(1)
    label_text = "Bars & Pubs"
    label_element = driver.find_element(By.XPATH, "//label[contains(., '{}')]".format(label_text))
    checkbox = label_element.find_element(By.XPATH, "preceding::input[1]")
    driver.execute_script("arguments[0].click();", checkbox)

    time.sleep(3)
    label_text = "Restaurants"
    label_element = driver.find_element(By.XPATH, "//label[contains(., '{}')]".format(label_text))
    checkbox = label_element.find_element(By.XPATH, "preceding::input[1]")
    driver.execute_script("arguments[0].removeAttribute('checked'); arguments[0].click();", checkbox)


In [105]:
try:
    text_to_find = "Show more"
    showMore = driver.find_element(By.XPATH, "//span[contains(text(), '{}')]".format(text_to_find))
    selectCheckbox1(showMore, driver) 

except NoSuchElementException:
    try:
        showMore = driver.find_element(By.XPATH, "//button[contains(@aria-controls, 'filter-expando-establishmentTypes')]")
        selectCheckbox2(showMore, driver)
    except NoSuchElementException:
        print("Error: Something went wrong on checkbox selection...")

time.sleep(3)   

In [106]:
def retrieveInfo(review, business_name):
    review_info = {}
    review_info['Business_name'] = business_name
    try:
        username_element = review.find_element(By.XPATH, ".//div[@class='info_text pointer_cursor']")
        username = username_element.text
    except NoSuchElementException:
        username = None
    review_info['Username'] = username
    try:
        date_element = review.find_element(By.XPATH, ".//span[@class='ratingDate']")
        review_date = date_element.get_attribute('title')
    except NoSuchElementException:
        review_date = None
    review_info['Review Date'] = review_date
    try:
        visit_date_element = review.find_element(By.XPATH, ".//div[@class='prw_rup prw_reviews_stay_date_hsx']")
        visit_date_text = visit_date_element.text
        visit_date = visit_date_text.replace("Date of visit:", "").strip()
    except NoSuchElementException:
        visit_date = None
    review_info['Visit Date'] = visit_date
    try:
        title_element = review.find_element(By.XPATH, ".//span[@class='noQuotes']")
        title = title_element.text
    except NoSuchElementException:
        title = None
    review_info['Review Title'] = title
        
    # Check if there is a "More" button to see full review text
    try:
        max_attempts = 3
        current_attempt = 1
        while current_attempt < max_attempts:
            try:
                # Define the locator for the clickable element within the parent element
                # clickable_locator = review.find_element(By.CLASS_NAME, 'ulBlueLinks')
                clickable_locator = review.find_element(By.XPATH, "//span[text() = 'More']")

                # Perform the click operation
                #driver.execute_script("arguments[0].click();", clickable_locator)
                clickable_locator.click()
                
                time.sleep(7)
                break  # Click successful, exit the loop

            except ElementClickInterceptedException:
                current_attempt += 1
                print(f"Attempt {current_attempt} failed. Retrying...")

    except NoSuchElementException:
        print("")
    except TimeoutException:
        print("Error: Timeout")
        
    # Extract review content
    content_element = review.find_element(By.XPATH, ".//p[@class='partial_entry']")
    content = content_element.text.strip()
    print(content)
    review_info['Review Text'] = content
    try:
        rating_element = review.find_element(By.XPATH, ".//span[contains(@class, 'ui_bubble_rating')]")
        rating_class = rating_element.get_attribute('class')
        rating = int(rating_class.split('bubble_')[-1]) / 10
    except NoSuchElementException:
        rating = None
    review_info['Rating'] = rating   

    return review_info 

In [107]:
def navigateReview(driver, review_data, business_name):
    try:
        # Find review elements
        reviews = driver.find_elements(By.CLASS_NAME, 'review-container')
        
        # Scrape review data
        for review in reviews:
            try:
                review_info = retrieveInfo(review, business_name)
                review_data.append(review_info)

            except ElementClickInterceptedException:
                time.sleep(5)
                review_info = retrieveInfo(review, business_name)
                review_data.append(review_info)
                print("ElementClickInterceptedException: navigateReview")
    except StaleElementReferenceException:
        print("StaleElementReferenceException: navigateReview")



In [108]:
def goToNextReviewPage(driver, review, business_name, counter):
    try:
        
        next_page_link = driver.find_element("xpath", f".//a[contains(@data-page-number, {counter})]")
            
        if next_page_link:
            next_page_link.click()
            time.sleep(3)

            # Update the review data by navigating to the new page
            navigateReview(driver, review, business_name)
            msg = ""

        # Check if the "Next" button is disabled
        elif 'disabled' in next_page_link.get_attribute('class'):
            msg = "No more pages."
            return msg

    except NoSuchElementException:
        # Handle the case when the "Next" button is not found
        msg = "No more pages."
        return msg

    # Return the updated review data and the status message
    return  msg

In [109]:
def getReviewInfo(driver, review_data):
    # Find the business name element
    business_name_element = driver.find_element(By.XPATH, "//h1[@data-test-target='top-info-header']")
    business_name = business_name_element.text

    print(f"Business Name: {business_name}")

    counter = 2
    while True:
        # Navigate to the current page and extract review data
        navigateReview(driver, review_data, business_name)
        # Go to the next page and update the review data
        msg = goToNextReviewPage(driver, review_data, business_name, counter)
        counter += 1
        # Check if there are no more pages
        if msg == "No more pages.":
            print(msg)
            break


In [110]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def  navigate(driver, businessCount, review_data):

    while True:
        try:
            print(f"business: {businessCount}")
            # Find the div element you want to click
            business_page = driver.find_element(By.CSS_SELECTOR, f"div[data-test='{businessCount}_list_item']")

            # Find the link element inside the div
            link_element = business_page.find_element(By.TAG_NAME, "a")

            # Perform the click action on the div element
            link_element.click()

            wait = WebDriverWait(driver, 10)  # Set a maximum wait time of 10 seconds
            wait.until(EC.number_of_windows_to_be(2))  # Wait until there are two open windows

            #Switch to the new window or tab
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(3)
            getReviewInfo(driver, review_data) 
            time.sleep(2)
            # Close the current window
            driver.close()

            # Switch back to the previous window
            driver.switch_to.window(driver.window_handles[0])
        except NoSuchElementException:
            return businessCount

        businessCount += 1


In [111]:
def goToPage(driver, businessCount, counter, review_data):
    try:
        next_page_link = driver.find_element("xpath", f".//a[contains(@aria-label, {counter})]")
    except NoSuchElementException:
        next_page_link = driver.find_element("xpath", f".//a[contains(@data-page-number, {counter})]")
           
    if next_page_link:
        next_page_link.click()
        time.sleep(3)
        businessCount = navigate(driver, businessCount, review_data)
    
    return businessCount
        

In [112]:
counter = 1
businessCount = 1
review_data = []

time.sleep(3)
businessCount = navigate(driver, businessCount, review_data)
print(f"businessCount: {businessCount}")

while True:
    # give the DOM time to load
    print(f"page: {counter}")
    time.sleep(3)
    try:
        businessCount = goToPage(driver, businessCount, counter, review_data)

    except ElementClickInterceptedException:
        # Retry the operation after waiting for some time
        time.sleep(5)
        continue
        
    except NoSuchElementException:
        break       
    counter += 1


# When all pages have been processed, quit the driver
# driver.quit()

#list that contains all the reviews form each business
print(review_data)

business: 1
Business Name: Albeta Mediterranean Bakery
Top quality products, exceptional customer service and tasty food. Coffee is also very good and one of my favorite choices.

A great place for breakfast! I really enjoyed the tortilla chicken and the fresh pomegranate juice.

The stuff was very helpful and kind! They also have a beg variety of pastries and the best coffee in town!

The best bakery in Thessaloniki. Highest quality of ingredients featuring baked goods to sandwiches & salads

It has everything, from delicious food to polite staff! It has variety of food, coffee and you can find kombucha (for the ones that love it and cannot find it easily in Thessaloniki). Don’t forget to check out the sweets as well :)

The best in town. Wonderful coffee, nice people, everything fresh and tasty. So nice to be there. Mornings get beautiful. 🙏🏻 💗💗💗💗

The cutest bakery located at the center of downtown Thessaloniki. Best quality and a variety of sweet and savoury pastries makes you want

KeyboardInterrupt: 